<a href="https://colab.research.google.com/github/metamatematico/hello-world/blob/master/Caso_de_estudio_Corrector_gramatical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caso de estudio: Corrector gramatical

In [ ]:
# Importación dela librería de SpaCy
from spacy import displacy
import spacy.cli
spacy.cli.download("es_core_news_sm")
pln_es = spacy.load("es_core_news_sm")

✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


In [ ]:
# Módulo para correguir errores de ortografía comunes
# Utilizando un diccionario

Diccionario = {'hola' : 'ola',  'favor': 'fabor', 'ver'    : 'ber',
               'hacer': 'acer', 'allá' : 'ayá',   'gracias': 'grasias',
               'saldo': 'zaldo'}

def ERRORES_COMUNES(Texto):
    Errores = 0
    Texto_PLN = pln_es(Texto)
    for token in Texto_PLN:
        for error, correcto in zip(Diccionario.values(), Diccionario.keys()):
            # Si nos encontramos una palabra mal escrita, la REEMPLAZAMOS
            if token.text.lower() == error:
                Texto = Texto.replace(error, '[' + correcto + ']')
                Errores += 1
    print(' * ' + str(Errores) + " errores de ortografía encontrados")
    return Texto

In [ ]:
# Módulo para correguir errores dentre singulares y plurales
# (Solo se aplica a nouns)

def TIEMPOS_VERBALES(Texto):
    Errores = 0
    Texto_PLN = pln_es(Texto.replace('[', '').replace(']', ''))
    lista = list(Texto_PLN)
    for i in range(len(lista)-1):
        # Se evalúa solo si el token es un NOUN
        if lista[i+1].pos == 'NOUN': 
            Value1 = lista[i].tag_.find('Number')
            Value2 = lista[i+1].tag_.find('Number')
            Palabra = lista[i+1].text
            if Value1 >= 0 and Value2 >= 0:
                # Si la etiqueta de "Number" para el token es singular, y la siguiente plural o al revés, es un error
                if lista[i].tag_[Value1 + 7 : Value1 + 11] == 'Sing' and lista[i+1].tag_[Value2 + 7 : Value2 + 11] == 'Plur':
                    Texto = Texto.replace(Palabra, '[' + Palabra[:len(Palabra)-1] + ']')
                    Errores += 1
                if lista[i].tag_[Value1 + 7 : Value1 + 11] == 'Plur' and lista[i+1].tag_[Value2 + 7 : Value2 + 11] == 'Sing':
                    Texto = Texto.replace(Palabra, '[' + Palabra + 's]')
                    Errores += 1
    print(' * ' + str(Errores) + " errores de sing / plur encontrados")
    return Texto

In [ ]:
# Módulo para detectar palabras repetidas

def PALABRA_REPETIDA(Texto):
    Errores = 0
    Texto_PLN = pln_es(Texto.replace('[', '').replace(']', ''))
    lista = list(Texto_PLN)
    for i in range(len(lista)-1):
        # Si la palabra es exactamente la misma que la siguioente, se trata de un error
        if(lista[i].text == lista[i+1].text):
            Texto = Texto.replace(lista[i].text + " " + lista[i].text, lista[i].text + ' []')
            Errores += 1
    print(' * ' + str(Errores) + " errores de palabras repetidas")
    return Texto

In [ ]:
# Módulo para detectar problemas con mayúsculas y minúsculas

def MAYUSC_MINUSC(Texto):
    Errores = 0
    Texto_PLN = pln_es(Texto)
    lista = list(Texto_PLN)
    for i in range(len(lista)):
        # Si las mayúsculas y minúsculas están mezcladas
        if str(lista[i].shape_).count("X")>1 and str(lista[i].shape_).count("x")>1:
            Texto = Texto.replace(lista[i].text, '[' + lista[i].text.lower() + ']')
            Errores += 1
        # Si hay una mayúscula y antes no hay un punto
        if str(lista[i].shape_).count("X")==1 and lista[i-1].text != '.' and i != 0:
            Texto = Texto.replace(lista[i].text, '[' + lista[i].text.lower() + ']')
            Errores += 1
    print(' * ' + str(Errores) + " errores de palaras con mayusc y minusc")
    return Texto

In [ ]:
# Modulo para detectar problemas con verbos auxiliares
# Necesitas / Debes / Puedes 

def VERBOS_AUXILIARES(Texto):
    Errores = 0
    Texto_PLN = pln_es(Texto.replace('[', '').replace(']', ''))
    lista = list(Texto_PLN)
    for i in range(len(lista)-1):
        # Si nos encontramos un Verbo auxiliar y después un verbo, se puede analizar
        if(lista[i].pos_ == 'AUX' and lista[i+1].pos_ == 'VERB'):
            Valor = lista[i+1].tag_.find('VerbForm')
            # Si el verbo después del auxiiar no está en infinitivo, es un error
            if lista[i+1].tag_[Valor + 9 : Valor + 12] != 'Inf':
                Texto = Texto.replace(lista[i+1].text, '[' + str(lista[i+1].lemma) + ']')
                Errores += 1
    print(' * ' + str(Errores) + " errores con verbos auxiliares")
    return Texto

In [ ]:
# Ingresamos el texto a evaluar y hacemos que sea evaluado
# por todos los módulos que ya definimos
Entrada = input("Texto para revisión: \n")
print('\n')

Entrada = ERRORES_COMUNES(Entrada)
Entrada = TIEMPOS_VERBALES(Entrada)
Entrada = PALABRA_REPETIDA(Entrada)
Entrada = MAYUSC_MINUSC(Entrada)
Entrada = VERBOS_AUXILIARES(Entrada)

print('\nTexto corregido:\n' + Entrada)

Texto para revisión: 
# ola señorita, me puede dar mi zaldo por fabor, grasias, lo que pasa pasa es que quiero Comprar una casas en la playa playa para que los niño jueguen en el patio. Mi PeRRo es muy juguetón, por eso me mudo a la CDMX. EL casero mE dijo: "debes pagado antes del viernes viernes".


 * 4 errores de ortografía encontrados
 * 0 errores de sing / plur encontrados
 * 3 errores de palabras repetidas
 * 3 errores de palaras con mayusc y minusc
 * 0 errores con verbos auxiliares

Texto corregido:
# [hola] señorita, me puede dar mi [saldo] por [favor], [gracias], lo que pasa [] es que quiero [comprar] una casas en la playa [] para que los niño jueguen en el patio. Mi [perro] es muy juguetón, por eso me mudo a la CDMX. EL casero [me] dijo: "debes pagado antes del viernes []".


In [ ]:
# Frases que puedes probar:
# ola, me das mi zaldo por fabor, grasias
# Quiero una casas en la playa
# los niño salieron a jugar al patio
# Palabras que se repiten repiten

# ola señorita, me puede dar mi zaldo por fabor, grasias, lo que pasa pasa es que quiero Comprar una casas en la playa playa para que los niño jueguen en el patio. Mi PeRRo es muy juguetón, por eso me mudo a la CDMX. EL casero mE dijo: "debes pagado antes del viernes viernes".

In [ ]:
# Se usan en los ejercicios
# .text
# .pos_
# .tag_
# .Shape_
# .lemma_